In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from pathlib import Path
from sqlalchemy import create_engine, text
import subprocess
pd.set_option('display.max_columns', None)

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
db_staging = os.getenv('DB_STAGING')
db_dwh = os.getenv('DB_DWH')

In [3]:
engine_read = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db_staging}')

In [4]:
heart_df = pd.read_sql('heart_data', con = engine_read)

In [5]:
heart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445132 entries, 0 to 445131
Data columns (total 42 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         445132 non-null  int64  
 1   SurveyDate                 445132 non-null  object 
 2   State                      445132 non-null  float64
 3   Sex                        445132 non-null  float64
 4   GeneralHealth              445129 non-null  float64
 5   PhysicalHealthDays         445127 non-null  float64
 6   MentalHealthDays           445129 non-null  float64
 7   LastCheckupTime            445129 non-null  float64
 8   PhysicalActivities         445130 non-null  float64
 9   SleepHours                 445129 non-null  float64
 10  RemovedTeeth               443769 non-null  float64
 11  HadHeartAttack             445128 non-null  float64
 12  HadAngina                  445130 non-null  float64
 13  HadStroke                  44

In [6]:
heart_df.head()

,id,SurveyDate,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,1,02032022,1.0,2.0,2.0,88.0,88.0,1.0,2.0,8.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,4.0,2.0,1.0,13.0,NaN,NaN,NaN,2.0,2.0,1.0,2.0,3.0,2.0,2.0
1,2,02042022,1.0,2.0,1.0,88.0,88.0,8.0,2.0,6.0,NaN,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,1.0,2.0,1.0,13.0,160.0,6804.0,2657.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0
2,3,02022022,1.0,2.0,2.0,2.0,3.0,1.0,1.0,5.0,NaN,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,1.0,2.0,1.0,8.0,157.0,6350.0,2561.0,2.0,2.0,2.0,2.0,7.0,2.0,1.0
3,4,02032022,1.0,2.0,1.0,88.0,88.0,1.0,1.0,7.0,NaN,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,14.0,165.0,6350.0,2330.0,2.0,2.0,1.0,1.0,4.0,2.0,2.0
4,5,02022022,1.0,2.0,4.0,2.0,88.0,1.0,1.0,9.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,1.0,1.0,1.0,5.0,157.0,5398.0,2177.0,1.0,2.0,2.0,1.0,4.0,2.0,2.0


## Cleaning and Transformation

In [7]:
def adjust_invalid_date(date_str):
    month = int(date_str[:2])
    day = int(date_str[2:4])
    year = int(date_str[4:])
    
    if month == 0 or month > 12:
        month = 1
    
    if day == 0:
        day = 1

    adjusted_date_str = f"{year}-{month:02d}-{day:02d}"
    return adjusted_date_str
    
heart_df['SurveyDate'] = heart_df['SurveyDate'].apply(adjust_invalid_date)
heart_df['SurveyDate'] = pd.to_datetime(heart_df['SurveyDate'])

In [8]:
heart_copy_df = heart_df.copy()

In [9]:
heart_copy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445132 entries, 0 to 445131
Data columns (total 42 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   id                         445132 non-null  int64         
 1   SurveyDate                 445132 non-null  datetime64[ns]
 2   State                      445132 non-null  float64       
 3   Sex                        445132 non-null  float64       
 4   GeneralHealth              445129 non-null  float64       
 5   PhysicalHealthDays         445127 non-null  float64       
 6   MentalHealthDays           445129 non-null  float64       
 7   LastCheckupTime            445129 non-null  float64       
 8   PhysicalActivities         445130 non-null  float64       
 9   SleepHours                 445129 non-null  float64       
 10  RemovedTeeth               443769 non-null  float64       
 11  HadHeartAttack             445128 non-null  float64 

In [10]:
for c in heart_df.columns:
    print("\n", heart_df[c].value_counts().sort_index())
    print("-"*30, end="\n")


 id
1         1
2         1
3         1
4         1
5         1
         ..
445128    1
445129    1
445130    1
445131    1
445132    1
Name: count, Length: 445132, dtype: int64
------------------------------

 SurveyDate
2022-01-04    26
2022-01-05    76
2022-01-06    64
2022-01-07    29
2022-01-08    26
              ..
2023-02-16     8
2023-02-17     2
2023-02-19     1
2023-02-20     1
2023-02-21     1
Name: count, Length: 412, dtype: int64
------------------------------

 State
1.0      4506
2.0      5865
4.0     10185
5.0      5309
6.0     10952
8.0      9365
9.0      9784
10.0     3987
11.0     3237
12.0    13393
13.0     9236
15.0     7747
16.0     6280
17.0     4056
18.0    10466
19.0     8949
20.0    11247
21.0     4023
22.0     5629
23.0    10646
24.0    16418
25.0    11029
26.0    10058
27.0    16821
28.0     4239
29.0     7438
30.0     7048
31.0     7473
32.0     3188
33.0     6757
34.0     8209
35.0     4758
36.0    17800
37.0     4505
38.0     4153
39.0    16487
40.0    

In [11]:
sleep_hours_median = heart_copy_df['SleepHours'].median()

GEN_HEALTH = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5
}

PHYS_MEN_HEALTH = {77: np.nan, 88: 0, 99: np.nan}

LAST_CHECKUP = {
    1: 1,
    2: 2,
    3: 3,
    4: 4
}

YES_NO_QUESTIONS = {1: 1, 2: 0}

SLEEP_TIME = lambda x: np.where(x > 24, sleep_hours_median, x)

TEETH_REMOVED = {
    1: 1,
    2: 2,
    3: 3,
    8: 4
}

DIABETES = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
}

SMOKER_STATUS = {
    1: 1,
    2: 2,
    3: 3,
    4: 4
}

ECIGARETTES = {
    1: 1,
    2: 2,
    3: 3,
    4: 4
}

RACE = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5
}

AGE_CATEGORY = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: 7,
    8: 8,
    9: 9,
    10: 10,
    11: 11,
    12: 12,
    13: 13
}

TETANUS = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
}

COVID = {
    1: 1,
    2: 2,
    3: 3
}

In [12]:
heart_copy_df['GeneralHealth'] = heart_copy_df['GeneralHealth'].map(GEN_HEALTH)
heart_copy_df['PhysicalHealthDays'] = heart_copy_df['PhysicalHealthDays'].replace(PHYS_MEN_HEALTH)
heart_copy_df['MentalHealthDays'] = heart_copy_df['MentalHealthDays'].replace(PHYS_MEN_HEALTH)
heart_copy_df['LastCheckupTime'] = heart_copy_df['LastCheckupTime'].map(LAST_CHECKUP)
heart_copy_df['PhysicalActivities'] = heart_copy_df['PhysicalActivities'].map(YES_NO_QUESTIONS)
heart_copy_df['SleepHours'] = heart_copy_df['SleepHours'].apply(SLEEP_TIME)
heart_copy_df['RemovedTeeth'] = heart_copy_df['RemovedTeeth'].map(TEETH_REMOVED)
heart_copy_df['HadHeartAttack'] = heart_copy_df['HadHeartAttack'].map(YES_NO_QUESTIONS)
heart_copy_df['HadAngina'] = heart_copy_df['HadAngina'].map(YES_NO_QUESTIONS)
heart_copy_df['HadStroke'] = heart_copy_df['HadStroke'].map(YES_NO_QUESTIONS)
heart_copy_df['HadAsthma'] = heart_copy_df['HadAsthma'].map(YES_NO_QUESTIONS)
heart_copy_df['HadSkinCancer'] = heart_copy_df['HadSkinCancer'].map(YES_NO_QUESTIONS)
heart_copy_df['HadCOPD'] = heart_copy_df['HadCOPD'].map(YES_NO_QUESTIONS)
heart_copy_df['HadDepressiveDisorder'] = heart_copy_df['HadDepressiveDisorder'].map(YES_NO_QUESTIONS)
heart_copy_df['HadKidneyDisease'] = heart_copy_df['HadKidneyDisease'].map(YES_NO_QUESTIONS)
heart_copy_df['HadArthritis'] = heart_copy_df['HadArthritis'].map(YES_NO_QUESTIONS)
heart_copy_df['HadDiabetes'] = heart_copy_df['HadDiabetes'].map(DIABETES)
heart_copy_df['DeafOrHardOfHearing'] = heart_copy_df['DeafOrHardOfHearing'].map(YES_NO_QUESTIONS)
heart_copy_df['BlindOrVisionDifficulty'] = heart_copy_df['BlindOrVisionDifficulty'].map(YES_NO_QUESTIONS)
heart_copy_df['DifficultyConcentrating'] = heart_copy_df['DifficultyConcentrating'].map(YES_NO_QUESTIONS)
heart_copy_df['DifficultyWalking'] = heart_copy_df['DifficultyWalking'].map(YES_NO_QUESTIONS)
heart_copy_df['DifficultyDressingBathing'] = heart_copy_df['DifficultyDressingBathing'].map(YES_NO_QUESTIONS)
heart_copy_df['DifficultyErrands'] = heart_copy_df['DifficultyErrands'].map(YES_NO_QUESTIONS)
heart_copy_df['SmokerStatus'] = heart_copy_df['SmokerStatus'].map(SMOKER_STATUS)
heart_copy_df['ECigaretteUsage'] = heart_copy_df['ECigaretteUsage'].map(ECIGARETTES)
heart_copy_df['ChestScan'] = heart_copy_df['ChestScan'].map(YES_NO_QUESTIONS)
heart_copy_df['RaceEthnicityCategory'] = heart_copy_df['RaceEthnicityCategory'].map(RACE)
heart_copy_df['AgeCategory'] = heart_copy_df['AgeCategory'].map(AGE_CATEGORY)
heart_copy_df['HeightInMeters'] = heart_copy_df['HeightInMeters'] / 100
heart_copy_df['WeightInKilograms'] = heart_copy_df['WeightInKilograms'] / 100
heart_copy_df['BMI'] = heart_copy_df['BMI'] / 100
heart_copy_df['AlcoholDrinkers'] = heart_copy_df['AlcoholDrinkers'].map(YES_NO_QUESTIONS)
heart_copy_df['HIVTesting'] = heart_copy_df['HIVTesting'].map(YES_NO_QUESTIONS)
heart_copy_df['FluVaxLast12'] = heart_copy_df['FluVaxLast12'].map(YES_NO_QUESTIONS)
heart_copy_df['PneumoVaxEver'] = heart_copy_df['PneumoVaxEver'].map(YES_NO_QUESTIONS)
heart_copy_df['TetanusLast10Tdap'] = heart_copy_df['TetanusLast10Tdap'].map(TETANUS)
heart_copy_df['HighRiskLastYear'] = heart_copy_df['HighRiskLastYear'].map(YES_NO_QUESTIONS)
heart_copy_df['CovidPos'] = heart_copy_df['CovidPos'].map(COVID)

In [13]:
def generate_random_date(date, checkup):
    start_date = pd.to_datetime('2012-01-01')
    end_date = pd.to_datetime('2012-01-01')
    if checkup == 1:
        start_date = date - pd.DateOffset(years=1)
        end_date = date
    elif checkup == 2:
        start_date = date - pd.DateOffset(years=2)
        end_date = date - pd.DateOffset(years=1)
    elif checkup == 3:
        start_date = date - pd.DateOffset(years=5)
        end_date = date - pd.DateOffset(years=2)
    else:
        start_date = pd.to_datetime('2012-01-01')
        end_date = date - pd.DateOffset(years=5)
    return start_date + (end_date - start_date) * np.random.rand()

heart_copy_df['DiagnoseDate'] = heart_copy_df.apply(lambda row: generate_random_date(row['SurveyDate'], row['LastCheckupTime']), axis=1)

In [14]:
for c in heart_copy_df.columns:
    print("\n", heart_copy_df[c].value_counts().sort_index())
    print("-"*30, end="\n")


 id
1         1
2         1
3         1
4         1
5         1
         ..
445128    1
445129    1
445130    1
445131    1
445132    1
Name: count, Length: 445132, dtype: int64
------------------------------

 SurveyDate
2022-01-04    26
2022-01-05    76
2022-01-06    64
2022-01-07    29
2022-01-08    26
              ..
2023-02-16     8
2023-02-17     2
2023-02-19     1
2023-02-20     1
2023-02-21     1
Name: count, Length: 412, dtype: int64
------------------------------

 State
1.0      4506
2.0      5865
4.0     10185
5.0      5309
6.0     10952
8.0      9365
9.0      9784
10.0     3987
11.0     3237
12.0    13393
13.0     9236
15.0     7747
16.0     6280
17.0     4056
18.0    10466
19.0     8949
20.0    11247
21.0     4023
22.0     5629
23.0    10646
24.0    16418
25.0    11029
26.0    10058
27.0    16821
28.0     4239
29.0     7438
30.0     7048
31.0     7473
32.0     3188
33.0     6757
34.0     8209
35.0     4758
36.0    17800
37.0     4505
38.0     4153
39.0    16487
40.0    

In [15]:
heart_copy_df = heart_copy_df[['id', 'SurveyDate', 'DiagnoseDate', 'State', 'Sex', 'GeneralHealth',
       'PhysicalHealthDays', 'MentalHealthDays', 'LastCheckupTime',
       'PhysicalActivities', 'SleepHours', 'RemovedTeeth', 'HadHeartAttack',
       'HadAngina', 'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'HeightInMeters', 'WeightInKilograms', 'BMI', 'AlcoholDrinkers',
       'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap',
       'HighRiskLastYear', 'CovidPos']]

In [16]:
heart_copy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445132 entries, 0 to 445131
Data columns (total 43 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   id                         445132 non-null  int64         
 1   SurveyDate                 445132 non-null  datetime64[ns]
 2   DiagnoseDate               445132 non-null  datetime64[ns]
 3   State                      445132 non-null  float64       
 4   Sex                        445132 non-null  float64       
 5   GeneralHealth              443934 non-null  float64       
 6   PhysicalHealthDays         434205 non-null  float64       
 7   MentalHealthDays           436065 non-null  float64       
 8   LastCheckupTime            436824 non-null  float64       
 9   PhysicalActivities         444039 non-null  float64       
 10  SleepHours                 445129 non-null  float64       
 11  RemovedTeeth               433772 non-null  float64 

In [17]:
heart_copy_df[['SurveyDate', 'LastCheckupTime', 'DiagnoseDate']].sample(5)

,SurveyDate,LastCheckupTime,DiagnoseDate
333891,2022-04-02,1.0,2021-05-21 22:33:48.654933938
105970,2023-01-02,1.0,2022-02-20 01:18:45.159629317
443282,2023-02-03,2.0,2021-03-02 18:17:57.521520197
303038,2022-04-05,1.0,2022-03-23 03:35:27.605535436
10136,2022-12-22,2.0,2021-06-26 08:58:47.636405752


## Creating the Dimension Tables

### state

In [18]:
STATE = {
    1: "Alabama",
    2: "Alaska",
    4: "Arizona",
    5: "Arkansas",
    6: "California",
    8: "Colorado",
    9: "Connecticut",
    10: "Delaware",
    11: "District of Columbia",
    12: "Florida",
    13: "Georgia",
    15: "Hawaii",
    16: "Idaho",
    17: "Illinois",
    18: "Indiana",
    19: "Iowa",
    20: "Kansas",
    21: "Kentucky",
    22: "Louisiana",
    23: "Maine",
    24: "Maryland",
    25: "Massachusetts",
    26: "Michigan",
    27: "Minnesota",
    28: "Mississippi",
    29: "Missouri",
    30: "Montana",
    31: "Nebraska",
    32: "Nevada",
    33: "New Hampshire",
    34: "New Jersey",
    35: "New Mexico",
    36: "New York",
    37: "North Carolina",
    38: "North Dakota",
    39: "Ohio",
    40: "Oklahoma",
    41: "Oregon",
    42: "Pennsylvania",
    44: "Rhode Island",
    45: "South Carolina",
    46: "South Dakota",
    47: "Tennessee",
    48: "Texas",
    49: "Utah",
    50: "Vermont",
    51: "Virginia",
    53: "Washington",
    54: "West Virginia",
    55: "Wisconsin",
    56: "Wyoming",
    66: "Guam",
    72: "Puerto Rico",
    78: "Virgin Islands"
}
state_df = pd.DataFrame.from_dict(STATE, orient='index', columns=['state'])
state_df.reset_index(names='code', inplace=True)
state_df.head()

,code,state
0,1,Alabama
1,2,Alaska
2,4,Arizona
3,5,Arkansas
4,6,California


### Gender

In [19]:
GENDER = {1: 'Male', 2: 'Female'}
sex_df = pd.DataFrame.from_dict(GENDER, orient='index', columns=['gender'])
sex_df.reset_index(names='id', inplace=True)
sex_df.head()

,id,gender
0,1,Male
1,2,Female


### General Health

In [20]:
GEN_HEALTH_TXT = {
    1: "Excellent",
    2: "Very good",
    3: "Good",
    4: "Fair",
    5: "Poor"
}
gen_health_df = pd.DataFrame.from_dict(GEN_HEALTH_TXT, orient='index', columns=['general_health'])
gen_health_df.reset_index(names='id', inplace=True)
gen_health_df.head()

,id,general_health
0,1,Excellent
1,2,Very good
2,3,Good
3,4,Fair
4,5,Poor


### Last Checkup Time

In [21]:
LAST_CHECKUP_TXT = {
    1: "Within past year (anytime less than 12 months ago)",
    2: "Within past 2 years (1 year but less than 2 years ago)",
    3: "Within past 5 years (2 years but less than 5 years ago)",
    4: "5 or more years ago"
}

last_checkup_df = pd.DataFrame.from_dict(LAST_CHECKUP_TXT, orient='index', columns=['last_checkup_time'])
last_checkup_df.reset_index(names='id', inplace=True)
last_checkup_df.head()

,id,last_checkup_time
0,1,Within past year (anytime less than 12 months ...
1,2,Within past 2 years (1 year but less than 2 ye...
2,3,Within past 5 years (2 years but less than 5 y...
3,4,5 or more years ago


### Removed Teeth

In [22]:
TEETH_REMOVED_TXT = {
    1: "1 to 5",
    2: "6 or more, but not all",
    3: "All",
    4: "None of them"
}

rem_teeth_df = pd.DataFrame.from_dict(TEETH_REMOVED_TXT, orient='index', columns=['removed_teeth'])
rem_teeth_df.reset_index(names='id', inplace=True)
rem_teeth_df.head()

,id,removed_teeth
0,1,1 to 5
1,2,"6 or more, but not all"
2,3,All
3,4,None of them


### Diabetes

In [23]:
DIABETES_TXT = {
    1: "Yes",
    2: "Yes, but only during pregnancy (female)",
    3: "No",
    4: "No, pre-diabetes or borderline diabetes",
}

diabetes_df = pd.DataFrame.from_dict(DIABETES_TXT, orient='index', columns=['diabetes_status'])
diabetes_df.reset_index(names='id', inplace=True)
diabetes_df.head()

,id,diabetes_status
0,1,Yes
1,2,"Yes, but only during pregnancy (female)"
2,3,No
3,4,"No, pre-diabetes or borderline diabetes"


### Smoking Status

In [24]:
SMOKING_STATUS_TXT = {
    1: "Current smoker - now smokes every day",
    2: "Current smoker - now smokes some days",
    3: "Former smoker",
    4: "Never smoked"
}

smoking_status_df = pd.DataFrame.from_dict(SMOKING_STATUS_TXT, orient='index', columns=['smoking_status'])
smoking_status_df.reset_index(names='id', inplace=True)
smoking_status_df.head()

,id,smoking_status
0,1,Current smoker - now smokes every day
1,2,Current smoker - now smokes some days
2,3,Former smoker
3,4,Never smoked


### E-Cigarette usage

In [25]:
ECIGARETTES_TXT = {
    1: "Never used e-cigarettes in my entire life",
    2: "Use them every day",
    3: "Use them some days",
    4: "Not at all (right now)"
}

e_cigarette_usage_df = pd.DataFrame.from_dict(ECIGARETTES_TXT, orient='index', columns=['e_cigarette_usage'])
e_cigarette_usage_df.reset_index(names='id', inplace=True)
e_cigarette_usage_df.head()

,id,e_cigarette_usage
0,1,Never used e-cigarettes in my entire life
1,2,Use them every day
2,3,Use them some days
3,4,Not at all (right now)


### Race Ethnicity

In [26]:
RACE_TXT = {
    1: "White only, Non-Hispanic",
    2: "Black only, Non-Hispanic",
    3: "Other race only, Non-Hispanic",
    4: "Multiracial, Non-Hispanic",
    5: "Hispanic"
}

race_ethnicity_category_df = pd.DataFrame.from_dict(RACE_TXT, orient='index', columns=['race_ethnicity_category'])
race_ethnicity_category_df.reset_index(names='id', inplace=True)
race_ethnicity_category_df.head()

,id,race_ethnicity_category
0,1,"White only, Non-Hispanic"
1,2,"Black only, Non-Hispanic"
2,3,"Other race only, Non-Hispanic"
3,4,"Multiracial, Non-Hispanic"
4,5,Hispanic


### Age Category

In [27]:
AGE_CATEGORY_TXT = {
    1: "Age 18 to 24",
    2: "Age 25 to 29",
    3: "Age 30 to 34",
    4: "Age 35 to 39",
    5: "Age 40 to 44",
    6: "Age 45 to 49",
    7: "Age 50 to 54",
    8: "Age 55 to 59",
    9: "Age 60 to 64",
    10: "Age 65 to 69",
    11: "Age 70 to 74",
    12: "Age 75 to 79",
    13: "Age 80 or older"
}

age_category_df = pd.DataFrame.from_dict(AGE_CATEGORY_TXT, orient='index', columns=['age_category'])
age_category_df.reset_index(names='id', inplace=True)
age_category_df.head()

,id,age_category
0,1,Age 18 to 24
1,2,Age 25 to 29
2,3,Age 30 to 34
3,4,Age 35 to 39
4,5,Age 40 to 44


### Tetanus

In [28]:
TETANUS_TXT = {
    1: "Yes, received Tdap",
    2: "Yes, received tetanus shot, but not Tdap",
    3: "Yes, received tetanus shot but not sure what type",
    4: "No, did not receive any tetanus shot in the past 10 years",
}

tetanus_last_10_tdap_df = pd.DataFrame.from_dict(TETANUS_TXT, orient='index', columns=['tetanus_last_10_tdap'])
tetanus_last_10_tdap_df.reset_index(names='id', inplace=True)
tetanus_last_10_tdap_df.head()

,id,tetanus_last_10_tdap
0,1,"Yes, received Tdap"
1,2,"Yes, received tetanus shot, but not Tdap"
2,3,"Yes, received tetanus shot but not sure what type"
3,4,"No, did not receive any tetanus shot in the pa..."


### Covid Pos

In [29]:
COVID_TXT = {
    1: "Yes",
    2: "No",
    3: "Tested positive using home test without a health professional"
}

covid_pos_df = pd.DataFrame.from_dict(COVID_TXT, orient='index', columns=['covid_pos'])
covid_pos_df.reset_index(names='id', inplace=True)
covid_pos_df.head()

,id,covid_pos
0,1,Yes
1,2,No
2,3,Tested positive using home test without a heal...


### Renaming Columns

In [30]:
heart_copy_df.columns

Index(['id', 'SurveyDate', 'DiagnoseDate', 'State', 'Sex', 'GeneralHealth',
       'PhysicalHealthDays', 'MentalHealthDays', 'LastCheckupTime',
       'PhysicalActivities', 'SleepHours', 'RemovedTeeth', 'HadHeartAttack',
       'HadAngina', 'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'HeightInMeters', 'WeightInKilograms', 'BMI', 'AlcoholDrinkers',
       'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap',
       'HighRiskLastYear', 'CovidPos'],
      dtype='object')

In [31]:
heart_copy_df.rename(columns={'SurveyDate':'survey_date',
                             'DiagnoseDate':'diagnose_date', 'State':'state_code',
                             'Sex':'gender_id', 'GeneralHealth':'general_health_id',
                             'PhysicalHealthDays':'physical_health_days', 'MentalHealthDays':'mental_health_days',
                             'LastCheckupTime':'last_checkup_time_id', 'PhysicalActivities':'physical_activities',
                             'SleepHours':'sleep_hours', 'RemovedTeeth':'removed_teeth_id',
                             'HadHeartAttack':'had_heart_attack', 'HadAngina':'had_angina',
                             'HadStroke':'had_stroke', 'HadAsthma':'had_asthma',
                             'HadSkinCancer':'had_skin_cancer', 'HadCOPD':'had_copd',
                             'HadDepressiveDisorder':'had_depressive_disorder', 'HadKidneyDisease':'had_kidney_disease',
                             'HadArthritis':'had_arthritis', 'HadDiabetes':'diabetes_status_id',
                             'DeafOrHardOfHearing':'deaf_or_hard_of_hearing', 'BlindOrVisionDifficulty':'blind_or_vision_difficulity',
                             'DifficultyConcentrating':'difficulty_concentrating', 'DifficultyWalking':'difficulty_walking',
                             'DifficultyDressingBathing':'difficulty_dressing_bathing', 'DifficultyErrands':'difficulty_errands',
                             'SmokerStatus':'smoking_status_id', 'ECigaretteUsage':'e_cigarette_usage_id',
                             'ChestScan':'chest_scan', 'RaceEthnicityCategory':'race_ethnicity_category_id',
                             'AgeCategory':'age_category_id', 'HeightInMeters':'height_in_meters',
                             'WeightInKilograms':'weight_in_kilograms', 'BMI':'bmi',
                             'AlcoholDrinkers':'alcohol_drinkers', 'HIVTesting':'hiv_testing',
                             'FluVaxLast12':'flu_vax_last_12', 'PneumoVaxEver':'pneumo_vax_ever',
                             'TetanusLast10Tdap':'tetanus_last_10_tdap_id', 'HighRiskLastYear':'high_risk_last_year',
                             'CovidPos':'covid_pos_id'
                             }, inplace=True)

In [32]:
heart_copy_df.columns

Index(['id', 'survey_date', 'diagnose_date', 'state_code', 'gender_id',
       'general_health_id', 'physical_health_days', 'mental_health_days',
       'last_checkup_time_id', 'physical_activities', 'sleep_hours',
       'removed_teeth_id', 'had_heart_attack', 'had_angina', 'had_stroke',
       'had_asthma', 'had_skin_cancer', 'had_copd', 'had_depressive_disorder',
       'had_kidney_disease', 'had_arthritis', 'diabetes_status_id',
       'deaf_or_hard_of_hearing', 'blind_or_vision_difficulity',
       'difficulty_concentrating', 'difficulty_walking',
       'difficulty_dressing_bathing', 'difficulty_errands',
       'smoking_status_id', 'e_cigarette_usage_id', 'chest_scan',
       'race_ethnicity_category_id', 'age_category_id', 'height_in_meters',
       'weight_in_kilograms', 'bmi', 'alcohol_drinkers', 'hiv_testing',
       'flu_vax_last_12', 'pneumo_vax_ever', 'tetanus_last_10_tdap_id',
       'high_risk_last_year', 'covid_pos_id'],
      dtype='object')

## Loading to MySQL

In [33]:
print(sqlalchemy.__version__)

2.0.0


In [34]:
engine_load = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db_dwh}')

In [35]:
with engine_load.begin() as connection:
    connection.execute(text("TRUNCATE TABLE f_data"))
    connection.execute(text("DELETE FROM d_state"))
    connection.execute(text("DELETE FROM d_gender"))
    connection.execute(text("DELETE FROM d_general_health"))
    connection.execute(text("DELETE FROM d_last_checkup_time"))
    connection.execute(text("DELETE FROM d_removed_teeth"))
    connection.execute(text("DELETE FROM d_diabetes_status"))
    connection.execute(text("DELETE FROM d_smoking_status"))
    connection.execute(text("DELETE FROM d_e_cigarette_usage"))
    connection.execute(text("DELETE FROM d_race_ethnicity_category"))
    connection.execute(text("DELETE FROM d_age_category"))
    connection.execute(text("DELETE FROM d_tetanus_last_10_tdap"))
    connection.execute(text("DELETE FROM d_covid_pos"))

In [36]:
state_df.to_sql("d_state", con = engine_load, if_exists= 'append', index= False)

54

In [37]:
sex_df.to_sql("d_gender", con = engine_load, if_exists= 'append', index= False)

2

In [38]:
gen_health_df.to_sql("d_general_health", con = engine_load, if_exists= 'append', index= False)

5

In [39]:
last_checkup_df.to_sql("d_last_checkup_time", con = engine_load, if_exists= 'append', index= False)

4

In [40]:
rem_teeth_df.to_sql("d_removed_teeth", con = engine_load, if_exists= 'append', index= False)

4

In [41]:
diabetes_df.to_sql("d_diabetes_status", con = engine_load, if_exists= 'append', index= False)

4

In [42]:
smoking_status_df.to_sql("d_smoking_status", con = engine_load, if_exists= 'append', index= False)

4

In [43]:
e_cigarette_usage_df.to_sql("d_e_cigarette_usage", con = engine_load, if_exists= 'append', index= False)

4

In [44]:
race_ethnicity_category_df.to_sql("d_race_ethnicity_category", con = engine_load, if_exists= 'append', index= False)

5

In [45]:
age_category_df.to_sql("d_age_category", con = engine_load, if_exists= 'append', index= False)

13

In [46]:
tetanus_last_10_tdap_df.to_sql("d_tetanus_last_10_tdap", con = engine_load, if_exists= 'append', index= False)

4

In [47]:
covid_pos_df.to_sql("d_covid_pos", con = engine_load, if_exists= 'append', index= False)

3

In [48]:
heart_copy_df.to_sql("f_data", con = engine_load, if_exists= 'append', index= False)

445132

## Versioning the Data with DVC

In [49]:
# dvc_df = heart_copy_df.copy()
# dvc_df.shape

In [50]:
filepath = Path('dvc_data/data/heart_data.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
heart_copy_df.to_csv(filepath, index=False)

In [51]:
filepath = Path('dvc_data/data/state.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
state_df.to_csv(filepath, index=False)

In [52]:
filepath = Path('dvc_data/data/gender.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
sex_df.to_csv(filepath, index=False)

In [53]:
filepath = Path('dvc_data/data/general_health.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
gen_health_df.to_csv(filepath, index=False)

In [54]:
filepath = Path('dvc_data/data/removed_teeth.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
rem_teeth_df.to_csv(filepath, index=False)

In [55]:
filepath = Path('dvc_data/data/diabetes_status.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
diabetes_df.to_csv(filepath, index=False)

In [56]:
filepath = Path('dvc_data/data/smoking_status.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
smoking_status_df.to_csv(filepath, index=False)

In [57]:
filepath = Path('dvc_data/data/e_cigarette_usage.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
e_cigarette_usage_df.to_csv(filepath, index=False)

In [58]:
filepath = Path('dvc_data/data/race_ethnicity_category.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
race_ethnicity_category_df.to_csv(filepath, index=False)

In [59]:
filepath = Path('dvc_data/data/age_category.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
age_category_df.to_csv(filepath, index=False)

In [60]:
filepath = Path('dvc_data/data/tetanus_last_10_tdap.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
tetanus_last_10_tdap_df.to_csv(filepath, index=False)

In [61]:
filepath = Path('dvc_data/data/covid_pos.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
covid_pos_df.to_csv(filepath, index=False)

In [62]:
filepath = Path('dvc_data/data/last_checkup.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
last_checkup_df.to_csv(filepath, index=False)